In [ ]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import json, os, re, shutil, sys, time
from importlib import reload
import collections, itertools
import unittest
from IPython.display import display, HTML

# NLTK for NLP utils and corpora
import nltk

# Pandas NumPy and TensorFlow
import pandas as pd
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from utils import utils_proj, vocabulary

from sklearn.model_selection import train_test_split

import lstm; reload(lstm)

% matplotlib inline

In [ ]:
names = ['timestamp','date','query','handle','message']
df = pd.read_csv(r'C:\Users\nconidas\Desktop\w266_final_project\sentiment140.csv',encoding='Latin1',names=names)
df['sentiment'] = df.index
#Instead of positive being 4 make positive 1
df.loc[df['sentiment'] == 4,'sentiment']=1


In [ ]:
df['message'].hist()

In [ ]:
#Just use 1000 rows while developing the code, full set takes too long
x_data = df['message'].head(1000).tolist()
y_data = df['sentiment'].head(1000).tolist()

#build the vocab
x_data = [i.split() for i in x_data]
flat_list = [item for sublist in x_data for item in sublist]
vocab = utils_proj.build_vocab(flat_list)


In [213]:
#Split into 70% train, 15% dev, 15% test

def train_test_dev(x,y,train_pct=.8,test_pct=.1,random_state=1):
    #Split into train and test
    X_train, X_test, y_train, y_test = (
        train_test_split(x, y, test_size=1-train_pct, random_state=random_state))
    #split test into dev and test
    X_dev, X_test, y_dev, y_test = (
        train_test_split(X_test, y_test, test_size=test_pct/(1-train_pct), random_state=random_state))
    
    return X_train,X_dev,X_test,y_train,y_dev,y_test


X_train,X_dev,X_test,y_train,y_dev,y_test = train_test_dev(x_data,y_data,.7,.15)

In [215]:
#train/dev/test words to ids
train_ids = utils_proj.preprocess_sentences(X_train,vocab) 
dev_ids = utils_proj.preprocess_sentences(X_dev,vocab)
test_ids = utils_proj.preprocess_sentences(X_test,vocab)

In [225]:
# Training parameters
max_time = 25
batch_size = 100
learning_rate = 0.01
num_epochs = 1

# Model parameters
model_params = dict(V=vocab.size, 
                    H=200, 
                    softmax_ns=200,
                    num_layers=2,
                    num_classes=2)

In [228]:
# Will print status every this many seconds
reload(lstm)
print_interval = 5

lm = lstm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)
    
    
    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        bi = utils.rnnlm_batch_generator(train_ids, batch_size, max_time)
        print("[epoch {:d}] Starting epoch {:d}".format(epoch, epoch))
        #### YOUR CODE HERE ####
        # Run a training epoch.
        run_epoch(lm, session, bi,True,
                  True,1.0,learning_rate)
        
        #### END(YOUR CODE) ####
        print("[epoch {:d}] Completed in {:s}".format(epoch, utils.pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpointls 
        saver.save(session, checkpoint_filename, global_step=epoch)
    
        ##
        # score_dataset will run a forward pass over the entire dataset
        # and report perplexity scores. This can be slow (around 1/2 to 
        # 1/4 as long as a full epoch), so you may want to comment it out
        # to speed up training on a slow machine. Be sure to run it at the 
        # end to evaluate your score.
      #  print("[epoch {:d}]".format(epoch), end=" ")
      #  score_dataset(lm, session, train_ids, name="Train set")
      #  print("[epoch {:d}]".format(epoch), end=" ")
      #  score_dataset(lm, session, test_ids, name="Test set")
      #  print("")
    print("[epoch {:d}]".format(epoch), end=" ")
    score_dataset(lm, session, train_ids, name="Train set")
    print("[epoch {:d}]".format(epoch), end=" ")
    score_dataset(lm, session, test_ids, name="Test set")
    print("")
    # Save final model
    saver.save(session, trained_filename)

NameError: name 'TF_SAVEDIR' is not defined